In [1]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sns
from keras.datasets import mnist, cifar10
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore")

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train= np.concatenate([x_train, x_test])
x_train.shape

(70000, 28, 28)

In [4]:
x_train = x_train.astype('float32')/255.
#x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:3]), 3))

In [5]:
xcut=x_train[:10000,:]
xcut.shape

(10000, 28, 28)

In [6]:
#clustering intra-image qui permet d'extraire les features
kmeans = KMeans(n_clusters=10)
list_clusters=[]
for i in range (len(xcut)):
    if (i%100)==0 :
        clear_output(wait=True)
        print((i/len(xcut))*100)
    list_clusters+=[kmeans.fit_predict(xcut[i])]

0.0


In [13]:
#fonction qui réordonne le numéro des clusters pour qu'il soit le meme dans chaque image
def same_clusters(list_clusters,k):
    x=np.array(list_clusters).shape
    a=0
    for i in range (x[0]):
        all_clusters=np.arange(k)
        used_clusters=[]
        for j in range (x[1]):
            if list_clusters[i][j] not in used_clusters:
                a=list_clusters[i][j]
                used_clusters.append(all_clusters[0])
                for k in range(x[1]):
                    if list_clusters[i][k]==a:
                        list_clusters[i][k]=all_clusters[0]
                    elif list_clusters[i][k]==all_clusters[0]:
                        list_clusters[i][k]=a
                all_clusters=np.delete(all_clusters,0)
    return list_clusters
    

In [14]:
this_is_the_end=np.array(same_clusters(list_clusters,90))

In [15]:
list_taille_clusters=np.arange(1,160,4)

In [16]:
#création des clusters d'images déjà clusterisé
list_clusters_ac=[]
j=0
for i in list_taille_clusters:
    clear_output(wait=True)
    print((j/len(list_taille_clusters))*100)
    ac = AgglomerativeClustering(n_clusters=i)
    clusters = ac.fit(this_is_the_end)
    list_clusters_ac+= [clusters.labels_]
    j+=1
    

97.5


In [19]:
list_clusters_kmeans=[]
for i in list_taille_clusters:
    kmeans = KMeans(n_clusters=i) #sqrt(n)=K
    clusters_kmeans= kmeans.fit_predict(this_is_the_end)
    list_clusters_kmeans+= [clusters_kmeans]

In [15]:
#sauvegarde des clusters
n = clusters_kmeans.max()
j = np.shape(clusters_kmeans)[0]
clusters2 =[[] for x in range(j)]
for i in range(j):
    clusters2[clusters_kmeans[i]].append(i)

clusters2 = np.array(clusters2)
clusters2= clusters2[:n]
np.save('.\Clusters\CofC_km_cifar10.npy', clusters2)

<ipython-input-15-a91d7851abf1>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  clusters2 = np.array(clusters2)


In [12]:
np.save('.\Clusters\CofC_km_cifar10.npy', clusters_kmeans)

In [26]:
for i in range (len(list_clusters_kmeans)):
    path=".\Clusters\CcKm"+list_taille_clusters[i].astype(str) +".npy "
    np.save(path, list_clusters_kmeans[i])